In [8]:
from nltk.translate.bleu_score import SmoothingFunction
from bert_score import score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import evaluate
rouge = evaluate.load('rouge')

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/home/csgrad/sunilruf/'
path = "/home/csgrad/sunilruf/emotion_chatbot/master/sunils_code/mistra/rank_64/rank_64_2/checkpoint-50"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, load_in_4bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
import json

json_file_path = '../../../ExTES.json'  # Change 'output.json' to your desired file name

# Write the list to the JSON file
with open(json_file_path, 'rb') as json_file:
    extes_data = json.load(json_file)

In [5]:
len(extes_data)

11178

In [32]:
extes_data[94]

{'scene': 'Body Image Concerns and Eating Disorders',
 'description': "I've recently gained weight and I'm really struggling with accepting my body. I have a trip to the beach coming up and I'm feeling anxious and self-conscious about wearing a swimsuit in front of my friends.",
 'content': [{'User': "I don't know what to do. I've tried dieting and exercising, but nothing seems to be working."},
  {'AI Strategy': '',
   'AI': "It sounds like you're going through a tough time. Can you tell me more about your feelings and concerns about your body image?"},
  {'User': "I just feel like I'm not attractive anymore. None of my clothes fit properly and I keep comparing myself to others."},
  {'AI Strategy': '',
   'AI': "I can understand how difficult this must be for you. It's common to have these feelings, but it's important to remember that beauty comes in all shapes and sizes. Have you considered talking to a therapist or counselor about your body image concerns?"},
  {'User': "I've thoug

In [20]:
if 'User' in extes_data[494]['content'][0]:
    print('yes')

In [11]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def generate_content(tokenizer, messages):
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
    tokenizer.pad_token = tokenizer.eos_token
    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100
    outputs = model.generate(
            input_ids=input_ids,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
    )
    candidate = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
    
    return candidate, input_ids, target_ids

In [ ]:
import random
random.seed(0)
for i in range(50):
    #num = random.randint(0, 11170)
    num = i
    data = extes_data[num]['content']
    print(num)
    messages = []
    bleu_1 = []
    rouge_1 = []
    rouge_2 = []
    rougle_l = []
    Bert_P = []
    Bert_R = []
    Bert_F1 = []
    for i in range(0, len(data)-1, 2):
        #print(data[i]['User'])
        if 'User' not in data[i]:
            i = i+1
        messages.append({"role": "user", "content":  data[i]['User']})
        
        input_ids = tokenizer.apply_chat_template(messages, tokenize=True, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")
        tokenizer.pad_token = tokenizer.eos_token
        
        outputs = model.generate(
                input_ids=input_ids,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.7,
                top_k=50,
                top_p=0.95
        )
        candidate = (tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("[/INST]")[-1])
        reference = data[i+1]['AI']
        
        references = reference.split(" ")
        candidates = candidate.split(" ")
        print("Reference: ", reference)
        print("Candidate: ", candidate)
        smoothie = SmoothingFunction().method4

        bleu_1.append(sentence_bleu(references, candidates, weights=(1, 0, 0, 0), smoothing_function=smoothie))
        results = rouge.compute(predictions=[candidate], references=[reference])
        rouge_1.append(results['rouge1'])   
        rouge_2.append(results['rouge2'])
        rougle_l.append(results['rougeL'])
        
        P, R, F1 = score([candidate], [reference], lang='en', model_type='bert-base-uncased')
        Bert_P.append(P)
        Bert_R.append(R)
        Bert_F1.append(F1)
        
        messages.append({"role": "assistant", "content": reference})

In [24]:



print("Bleu1 average: ", sum(bleu_1)/len(bleu_1))
print("Rouge1 average: ", sum(rouge_1)/len(rouge_1))
print("Rouge2 average: ", sum(rouge_2)/len(rouge_2))
print("RougeL average: ", sum(rougle_l)/len(rougle_l))
print("Bert_P average: ", sum(Bert_P)/len(Bert_P))
print("Bert_R average: ", sum(Bert_R)/len(Bert_R))
print("Bert_F1 average: ", sum(Bert_F1)/len(Bert_F1))

print("Bleu1 max: ", max(bleu_1))
print("Rouge1 max: ", max(rouge_1))
print("Rouge2 max: ", max(rouge_2))
print("RougeL max: ", max(rougle_l))
print("Bert_P max: ", max(Bert_P))
print("Bert_R max: ", max(Bert_R))
print("Bert_F1 max: ", max(Bert_F1))


Bleu1 average:  0.014285714285714289
Rouge1 average:  0.33006797281087563
Rouge2 average:  0.09999999999999999
RougeL average:  0.2526989204318273
Bert_P average:  tensor([0.5686])
Bert_R average:  tensor([0.5642])
Bert_F1 average:  tensor([0.5661])
Bleu1 max:  0.028571428571428577
Rouge1 max:  0.4634146341463415
Rouge2 max:  0.19999999999999998
RougeL max:  0.34146341463414637
Bert_P max:  tensor([0.6851])
Bert_R max:  tensor([0.6523])
Bert_F1 max:  tensor([0.6683])


In [31]:
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize

import nltk
nltk.download('averaged_perceptron_tagger')
def extract_head_nouns_verbs(text):
    # Tokenize the text into words
    words = word_tokenize(text)
    
    # Perform part-of-speech tagging
    tagged_words = pos_tag(words)
    
    # Extract head nouns and verbs
    head_nouns_verbs = [word for word, tag in tagged_words if tag.startswith('NN') or tag.startswith('VB')]
    
    return head_nouns_verbs

# Example usage
#text = "I went to the park and played with my dog."
text = "No problem at all! Take care of yourself and don't hesitate to ask for help when needed. Have a fantastic day!"
head_nouns_verbs = extract_head_nouns_verbs(text)
print(" ".join(head_nouns_verbs))


problem Take care yourself do hesitate ask help needed Have day


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

def recursive_summary(text, model, tokenizer, max_length=512, threshold=100):
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=max_length, truncation=True)
    outputs = model.generate(inputs, max_length=max_length, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    if len(summary) <= threshold:
        return summary
    else:
        return recursive_summary(summary, model, tokenizer, max_length, threshold)

# Load the T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# Example usage
text = "This is a long piece of text that needs to be summarized recursively. It contains multiple paragraphs and sentences. The goal is to generate a concise summary of the entire text."
summary = recursive_summary(text, model, tokenizer)
print(summary)


In [2]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.1/920.1 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 34.0 MB/s eta 0:00:00
     

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [6]:
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

def extract_head_nouns(text):
    # Parse the text with spaCy
    doc = nlp(text)
    
    # Extract head nouns
    head_nouns = []
    for token in doc:
        if token.pos_ == "NOUN" and token.head.pos_ != "NOUN":
            head_nouns.append(token.text)
    
    return head_nouns

# Example usage
user_response = "I am very sad"
head_nouns = extract_head_nouns(user_response)
print(head_nouns)


[]


In [7]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

text = "This is a sample sentence with some stop words, such as the, is, and with."
filtered_text = remove_stopwords(text)
print(filtered_text)


sample sentence stop words, the, is, with.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/csgrad/sunilruf/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
